# Building Chatbot Using Attention Based Neural Networks

In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import absolute_import
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [7]:
corpus = "movie_corpus"
corpus_name = "movie_corpus"
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

In [8]:
with open(datafile, 'rb') as file:
    lines = file.readlines()
    
for line in lines[:3]:
    print(str(line) + '\n')

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"

b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"

b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"



In [9]:
PAD_token = 0 
SOS_token = 1
EOS_token = 2

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addWord(self, w):
        if w not in self.word2index:
            self.word2index[w] = self.num_words
            self.word2count[w] = 1
            self.index2word[self.num_words] = w
            self.num_words += 1
        else:
            self.word2count[w] += 1        
        
    def addSentence(self, sent):
        for word in sent.split(' '):
            self.addWord(word)


    def trim(self, min_cnt):
        if self.trimmed:
            return
        self.trimmed = True

        words_to_keep = []

        for k, v in self.word2count.items():
            if v >= min_cnt:
                words_to_keep.append(k)

        print('Words to Keep: {} / {} = {:.2%}'.format(
            len(words_to_keep), len(self.word2index), len(words_to_keep) / len(self.word2index)
        ))

        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

        for w in words_to_keep:
            self.addWord(w)

In [10]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def cleanString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def readVocs(datafile, corpus_name):
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    pairs = [[cleanString(s) for s in l.split('\t')] for l in lines]
    voc = Vocabulary(corpus_name)
    return voc, pairs

def filterPair(p, max_length):
    return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length

def filterPairs(pairs, max_length):
    return [pair for pair in pairs if filterPair(pair, max_length)]

def loadData(corpus, corpus_name, datafile, max_length):
    voc, pairs = readVocs(datafile, corpus_name)
    print(str(len(pairs)) + " Sentence pairs")
    pairs = filterPairs(pairs,max_length)
    print(str(len(pairs))+ " Sentence pairs after trimming")
    for p in pairs:
        voc.addSentence(p[0])
        voc.addSentence(p[1])
    print(str(voc.num_words) + " Distinct words in vocabulary")
    return voc, pairs

In [11]:
max_length = 10 
voc, pairs = loadData(corpus, corpus_name, datafile, max_length)

221282 Sentence pairs
64271 Sentence pairs after trimming
18008 Distinct words in vocabulary


In [12]:
print("Example Pairs:")
for pair in pairs[-10:]:
    print(pair)

Example Pairs:
['four', 'three minutes to go !']
['three minutes to go !', 'yes .']
['another fifteen seconds to go .', 'do something ! stall them !']
['yes sir name please ?', 'food !']
['food !', 'do you have a reservation ?']
['do you have a reservation ?', 'food ! !']
['grrrhmmnnnjkjmmmnn !', 'franz ! help ! lunatic !']
['what o clock is it mr noggs ?', 'eleven o clock my lorj']
['stuart ?', 'yes .']
['yes .', 'how quickly can you move your artillery forward ?']


In [13]:
def removeRareWords(voc, all_pairs, minimum):
    voc.trim(minimum)
    
    pairs_to_keep = []
    
    for p in all_pairs:
        keep = True
        
        for word in p[0].split(' '):
            if word not in voc.word2index:
                keep = False
                break
        for word in p[1].split(' '):
            if word not in voc.word2index:
                keep = False
                break

        if keep:
            pairs_to_keep.append(p)

    print("Trimmed from {} pairs to {}, {:.2%} of total".format(len(all_pairs)\
        , len(pairs_to_keep), len(pairs_to_keep)/ len(all_pairs)))
    return pairs_to_keep


minimum_count = 3
pairs = removeRareWords(voc, pairs, minimum_count)

Words to Keep: 7823 / 18005 = 43.45%
Trimmed from 64271 pairs to 53165, 82.72% of total


In [14]:
def indexFromSentence(voc, sent):
    return [voc.word2index[w] for w in sent.split(' ')] + [EOS_token]

def zeroPad(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def inputVar(l, voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    padList = zeroPad(indexes_batch)
    padTensor = torch.LongTensor(padList)
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    return padTensor, lengths

def getMask(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def outputVar(l, voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPad(indexes_batch)
    mask = torch.BoolTensor(getMask(padList))
    padTensor = torch.LongTensor(padList)
    return padTensor, mask, max_target_len

def batch2Train(voc, batch):
    batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    
    input_batch = []
    output_batch = []
    
    for p in batch:
        input_batch.append(p[0])
        output_batch.append(p[1])
        
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    
    return inp, lengths, output, mask, max_target_len

In [23]:
test_batch_size = 5
batches = batch2Train(voc, [random.choice(pairs) for _ in range(test_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("Input:")
print(input_variable)

print("Target:")
print(target_variable)

print("Mask:")
print(mask)

Input:
tensor([[   7,   50,   36,   79,   56],
        [ 112,   53, 1082,    4,  827],
        [ 118,  598,   56,   36,    4],
        [  40,   92,   12,   37,    2],
        [ 289,    7, 1295,   79,    0],
        [  83,  278,    4,    4,    0],
        [ 916,    6,    2,    2,    0],
        [   6,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
Target:
tensor([[  25,   25,   36,   36,  122],
        [ 112,   25,  148,   37,    6],
        [ 118,  132,  380,    9,    2],
        [  40, 2575,    9, 1053,    0],
        [ 289,  170, 2201,    4,    0],
        [   7,    4,  349,    2,    0],
        [ 916,    2,    2,    0,    0],
        [   4,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
Mask:
tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  T

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [25]:
class Attn(nn.Module):
    def __init__(self, hidden_size):
        super(Attn, self).__init__()
        self.hidden_size = hidden_size

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        attn_energies = self.dot_score(hidden, encoder_outputs)
        attn_energies = attn_energies.t()
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [26]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(DecoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(2 * hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.gru(embedded, last_hidden)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

In [27]:
def NLLMaskLoss(inp, target, mask):
    TotalN = mask.sum()
    CELoss = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = CELoss.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, TotalN.item()

In [28]:
MAX_LENGTH = 10
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_TF = True if random.random() < teacher_forcing_ratio else False

    if use_TF:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = NLLMaskLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = NLLMaskLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    loss.backward()

    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [29]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer,\
               decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, \
               save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    training_batches = [batch2Train(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    print('Starting ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    print("Beginning Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent done: {:.1f}%; Mean loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [30]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        return all_tokens, all_scores

In [31]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=max_length):
    indices = [indexFromSentence(voc, sentence)]
    lengths = torch.tensor([len(indexes) for indexes in indices])
    input_batch = torch.LongTensor(indices).transpose(0, 1)
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    tokens, scores = searcher(input_batch, lengths, max_length)
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def runChatBot(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            input_sentence = input('> ')
            if input_sentence == 'quit': break
            input_sentence = cleanString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Response:', ' '.join(output_words))

        except KeyError:
            print("Error: Unknown Word")

In [32]:
model_name = 'chatbot_model'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.15
batch_size = 64

loadFilename = None
checkpoint_iter = 4000

if loadFilename:
    checkpoint = torch.load(loadFilename)
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(voc.num_words, hidden_size)

if loadFilename:
    embedding.load_state_dict(embedding_sd)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = DecoderRNN(embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [33]:
save_dir = './'

clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0

epochs = 4000

print_every = 1
save_every = 500

encoder.train()
decoder.train()

print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, epochs, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Starting ...
Beginning Training...
Iteration: 1; Percent done: 0.0%; Mean loss: 8.9581
Iteration: 2; Percent done: 0.1%; Mean loss: 8.8519
Iteration: 3; Percent done: 0.1%; Mean loss: 8.6399
Iteration: 4; Percent done: 0.1%; Mean loss: 8.4067
Iteration: 5; Percent done: 0.1%; Mean loss: 8.0479
Iteration: 6; Percent done: 0.1%; Mean loss: 7.4559
Iteration: 7; Percent done: 0.2%; Mean loss: 6.9914
Iteration: 8; Percent done: 0.2%; Mean loss: 6.8347
Iteration: 9; Percent done: 0.2%; Mean loss: 6.7042
Iteration: 10; Percent done: 0.2%; Mean loss: 6.3910
Iteration: 11; Percent done: 0.3%; Mean loss: 6.3344
Iteration: 12; Percent done: 0.3%; Mean loss: 5.9856
Iteration: 13; Percent done: 0.3%; Mean loss: 5.7622
Iteration: 14; Percent done: 0.4%; Mean loss: 5.4934
Iteration: 15; Percent done: 0.4%; Mean loss: 5.5479
Iteration: 16; Percent done: 0.4%; Mean loss: 5.2254
Iteration: 17; Percent done: 0.4%; Mean loss: 5.1725
Iteration: 18; Percent done: 0

In [34]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

In [35]:
runChatBot(encoder, decoder, searcher, voc)

> Hi
Response: hi . you ? a minute ?
> how are you.
Response: i m fine . you ? it ?
> i dont like you
Error: Unknown Word
> my name is sahil
Error: Unknown Word
> why unkown word?
Error: Unknown Word
> hi
Response: hi . you ? a minute ?
> im good
Response: you re losing your own good night .
> ohh really
Response: i m sorry . you ? it ?
> haha
Error: Unknown Word
> im happy
Response: yes . you to do that ?


KeyboardInterrupt: ignored